Since I have 3 graveyards of tested models in my AI primer repository and the process of working towards the 100% 100% was rather hectic and all over the place, I am writing a new file to showcase progress from recollection. 

I have found a post by Andrej Karpathy that would have helped me http://karpathy.github.io/2019/04/25/recipe/. Though this is just one of many things I've found during the journey.

## Get that GPU working

First challenge was to install TensorFlow locally in a way so it would recognise the GPU in my computer and so the training would become as fast as possible. Took me longer than I'd like to admit. Long story short, https://www.tensorflow.org/install/pip is the way to go.

## Get the data

I looked at the data. Standardized it, checked the shapes of the arrays I was going to be working with, plotted some of the images. 

In [ ]:
data = np.load('flatland_train.npz')
X = data['X']
y = data['y']

y[y != 0] -= 2
X = X / 255.

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
Counter(y)

In [ ]:
for i in range(30):
    ax = plt.subplot(5,6,i+1)
    
    ax.axis('off')
    plt.imshow(X[i],cmap='gray')

Worthy to notice, I should learn to spend more time on getting to know the data. According to Karpathy at least. On the other hand, this dataset was clear and clean enough so that I could skip this crucial step. You could say the dataset, as in many instances in learning, was spoonfed to me.

## Build the model

I knew right from the start that I was going to have to augment the data to arrive at a perfect result, but I wanted to first see how far I could get without touching the data and only building the model.

I came into the project not even knowing what CNNs were, so naturally, I started off with just regular deep layers. The architecture:

Dense 2000 relu

Dropout 0.2

Dense 1000 relu

Dropout 0.2

Dense 500 relu

Dropout 0.2

Dense 250 relu

Dropout 0.2

Dense out softmax

That's fat. 7,630,005 weights. Bad results. I recognised I had to dive into CNNs. Before CNNs I wanted to better understand neural networks in general. So I did. [3b1b series on neural networks](https://www.youtube.com/watch?v=aircAruvnKk&list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi) was a great starting point for a deeper understanding. 

Some CNN reading: https://deeplizard.com/learn/video/YRhxdVk_sIs, http://colah.github.io/, https://developers.google.com/machine-learning/guides/rules-of-ml

More than working on the project in the first stages, I did reading on convolutional layers and pooling. Then I came back to building my own little thing. 

Still ignorant to the details of building an architecture, I brute-forced a starting model:

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=[50, 50, 1]))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(keras.layers.Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(keras.layers.Dense(10, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=3e-4),
              metrics=["accuracy"])
model.summary()

#loss = model.fit(X, y, epochs=8, batch_size=16, validation_split=0.2)

#pd.DataFrame(loss.history).plot()

Train set accuracy 98.33% Test set (simple) accuracy 99.54% Test set (adv.) accuracy 85.39%

Without any data augmentation we arrived at a decent result. Still though. 2,435,370 weights. Not the most lightweight model. Main takeaway from this was that convolutional layers work and how they work. Pooling was a new discovery for me as well. Good insight.

Looking from the current perspective into that recent model - dropout after convolutional layers seems wrong. Many convolutional layers without pooling inbetween seems wrong. Extremely small batch size seems sub-optimal.

## Augment that data!

First reference article:
https://arxiv.org/pdf/1003.0358.pdf

I was going to work with data augmented in batches. So I could make a bigger dataset. Fix the shape of the dataset, so that the model would eat it up nicely. 

In [ ]:
X_turbo=np.concatenate((X,X,X,X,X,X))
y_turbo=np.concatenate((y,y,y,y,y,y))
X_turbo = np.expand_dims(X_turbo, axis=3)

Long story short: [ImageDataGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator). Don't forget to apply it.

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=45,
    width_shift_range=0.55,
    height_shift_range=0.55,
    zoom_range = [0.35, 1.65],
    shear_range = 13,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')

I just learned to augment my pictures. Neat. The results?

(Don't forget to apply the augmentation when building the model by using datagen.flow as seen in the model)

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=[50, 50, 1]))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(keras.layers.Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(keras.layers.Dense(10, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=3e-4),
              metrics=["accuracy"])
model.summary()

#loss = model.fit(datagen.flow(X_aug,y,batch_size=8), epochs=20)

Train set accuracy 98.52% Test set (simple) accuracy 100.00% Test set (adv.) accuracy 98.30%

Woah! That was easy. I thought I was champion. Well turns out I was not. 272,682 weights. Pretty light. I liked this. 

I look at it now and recognise that the amount of naked convolutional layers is disgusting and the model is faaar from perfect. But this was the moment where I started to work for the last few percent to arrive at 100% 100%. And so the main part of the work and reading came after this moment. I would say that working towards 100 100 after this moment was the most insightful and useful as a student. I now started turning my eye to different neural network architectures. By far the most interesting topic for me in the project. And so began my journey of self-discoveries again.

From here I was reading - improving - reading - improving and so on and so forth until I had a reasonable model.

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=[50, 50, 1]))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(keras.layers.Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(keras.layers.Dense(10, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=3e-4),
              metrics=["accuracy"])
model.summary()

#loss = model.fit(datagen.flow(X_turbo,y_turbo,batch_size=4), epochs=70)

Boy is getting the right idea. Pooling after each few convolutional layers is a standard procedure in most (if not all(?)) architectures.

The recent model did not work out. Or at least gave worse results when posting on the link. Made me go back to my old ways of brute-forcing it: "Wait. This doesn't work. I want to go back to the old model." So I did.

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=[50, 50, 1]))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(keras.layers.Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(keras.layers.Dense(10, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=3e-4),
              metrics=["accuracy"])
model.summary()

#loss = model.fit(datagen.flow(X_turbo,y_turbo,batch_size=128), epochs=100)

Train set accuracy 98.52% Test set (simple) accuracy 100.00% Test set (adv.) accuracy 99.95%

Damn. This must really be a good model. It wasn't a good model. Well it wasn't awful because it worked. On the other hand, I am now glad that this model did not give me 100% 100% immediately. Once again, not getting 100 100 pushed me to learn and I am now extremely glad it did. If I had gotten the 100 100 with this weird model, I would have stopped at this point.

After this I went on to decrease my result for a better architecture in return. I started trying to copy other architectures and looking for inspiration online. I started playing around with different architectures. 

Some reading: https://theaisummer.com/cnn-architectures/, https://paperswithcode.com/sota/image-classification-on-imagenet, https://github.com/pranshushah/cifar-10_resnet_tensorflow

Worthy to note, I was inspired to copy resnet. But I couldn't get residual connections to work in keras and there is not a predefined residual connection layer or option in keras so I had to go back to the simple ways for the time being. 

## The evolution

Since I have changed the models in the graveyards so much, I don't have an exact evolution of them from the most recent one shown above to the final one. I will try to write what changes to the architecture worked and the thought process behind them.

### Remove dense layers

Dense layers add in too much weight to the model and do little of the job. I found out pretty quickly by experimenting that dense layers are absolutely redundant for this task. In fact, in this case, they actually make the model worse. Removed them completely and started working with convolutional layers only.

### Double filter number for conv layers after each pooling

Following the usual shape of many top architectures, I realised the amount of filters should go double up after each pooling. Or at least the amount of filters should always go up, not down. The final model, as shown below, goes 2x16 -> 2x32 -> 2x64 between pooling. I usually enjoyed 2x32 -> 2x64 -> 1x128 as a nice balance between amount of weights and power of model.

### The majority of work is done in the first layers

We are classifying polygons. If we think about what we learned about convolutional networks, we may realise that the most basic shapes in images are found out by the first layers of the network. Therefore such logic and the simplicity of the objects we are classifying made me realise I should put emphasis on the first layers of the network.

### Size up those first layer filters!

The importance of first layers made me start playing with the size of their filters. I quickly found that increasing the size of the first layer filters improves the model manyfold. Even one single layer neural network with sufficiently big filter size goes up in accuracy quite fast. 

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=(13, 13), activation='relu', input_shape=[50, 50, 1]))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(keras.layers.Dense(10, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=3e-4),
              metrics=["accuracy"])
model.summary()

Something like this usually does surprisingly well for the task. What was left now was to use this as a foundation to build something proper.

First - the beefy filter layer likes a partner layer. The partner could have a big filter size as well (like 9x9 in the case above). But later I settled for just the regular size for that second filter - (3x3). MaxPooling. 2 regular conv layers. Maxpooling. 1 or 2 regular conv layers. Maxpooling. Flatten. Output dense layer.

This was more or less the architecture I found to be the best. Although I would experiment by adding a dense layer, playing with various layers, normalisation, dropout, changing batch size, epoch number, WORKING WITH LEARNING RATE, I would always come back to the simple architecture:

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=(13, 13), activation='relu', input_shape=[50, 50, 1]))

model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(keras.layers.Dense(10, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=3e-4),
              metrics=["accuracy"])
model.summary()

### GlobalPooling trained models don't work with the link

For some reason if the model is trained using GlobalPooling after all the conv layers, the [link](https://us-central1-aiprimer.cloudfunctions.net/flatland?model_link=) breaks. I don't know if I did something wrong by trying globalpooling or what the case was. Model trains fine. Link doesn't seem to take such a model in.

### Learning rate

Although 3e-4 for the Adam optimizer does really well, I have played my own game with the learning rate. Through various tries I have come to the conclusion that a learning rate defined by an exponential decay scheduler works wonders for my case:

In [ ]:
initial_learning_rate = 4e-4
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.98,
    staircase=True)

Though I know now that learning rate is probably the most important hyperparameter in a model. There is always work to be done to tweak the learning rate.

## Final model

I got really close many times. I knew it was more or less a matter of luck in tweaking the numbers more or less to get the wanted hundreds. What came was many different attempts with small changes. The following model was the one that got the perfect result and is as lightweight as I could find it to be:

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(16, kernel_size=(9, 9), activation='relu', input_shape=[50, 50, 1]))

model.add(keras.layers.Conv2D(16, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(keras.layers.Dense(10, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
              metrics=["accuracy"])
model.summary()

Train set accuracy 98.52% Test set (simple) accuracy 100.00% Test set (adv.) accuracy 100.00%

952 KB. 75,514 weights. 

Lovely. 

As of now I am, still building a smaller model yet. I call it Featherweight. 

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size=(9, 9), activation='relu', input_shape=[50, 50, 1]))

model.add(keras.layers.Conv2D(8, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(16, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.Conv2D(16, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(keras.layers.Dense(10, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
              metrics=["accuracy"])
model.summary()

Train set accuracy 98.50% Test set (simple) accuracy 99.97% Test set (adv.) accuracy 99.97%

301KB. 19,906 weights.

Sweet.